In [9]:
# Import all stuff we need
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.pylab as pylab
import warnings
warnings.filterwarnings('ignore')

from brian import Network, Equations, NeuronGroup, Connection,\
    SpikeMonitor, raster_plot, StateMonitor, clear, reinit
from brian.stdunits import ms, mV

%matplotlib inline
pylab.rcParams['figure.figsize'] = 12, 8  # changes figure size (width, height) for larger images

In [10]:
clear(True, True) 
reinit() 

In [21]:
# The equations defining our neuron model
eqs_string_e = ''' dV/dt = 1.0/tau_e * (mu - V) + I_syn : volt
             mu : volt
             tau : ms
             I_syn : volt/ms
             '''

eqs_string_i = ''' dV/dt = 1.0/tau_i * (mu - V) + I_syn : volt
             mu : volt
             tau : ms
             I_syn : volt/ms
             '''

In [12]:
# Our model parameters
tau_e = 15*ms # membrane time constant (for excitatory synapses)
tau_i = 10*ms # membrane time constant (for inhibitory synapses)
tau_syn_e = 3*ms # exc synaptic time constant tau2 in paper
tau_syn_i = 2*ms # inh synaptic time constant tau2 in paper
syn_delay = 1*ms # synaptic delay between two neurons tau1 in paper
V_th = -50*mV # firing threshold
V_reset = -65*mV # reset potential
refr_period = 5*ms # absolute refractory period

# Our parameters for the bias mu
mu_e = np.array([1.1, 1.2]) # bias for excitatory neurons
mu_i = np.array([1.0, 1.05]) # bias for inhibitory neurons

# Number of neurons
N_e = 4000 # number of exc neurons
N_i = 1000 # number of inh neurons

# Connection probability
conn_prob_ee = 0.2
conn_prob_ii = conn_prob_ei = conn_prob_ie = 0.5

# Duration of our simulation
duration = 2000*ms

In [22]:
# Let's create an equation object from our string and parameters
model_eqs_e = Equations(eqs_string_e,
                     tau_e=tau_e)
model_eqs_i = Equations(eqs_string_i,
                     tau_i=tau_i)

In [23]:
neurons_e = NeuronGroup(N=N_e,
                        model=model_eqs_e,
                        threshold=V_th,
                        reset=V_reset,
                        refractory=refr_period,
                        freeze = True,
                        method='Euler',
                        compile=True)

neurons_i = NeuronGroup(N=N_i,
                        model=model_eqs_i,
                        threshold=V_th,
                        reset=V_reset,
                        refractory=refr_period,
                        freeze = True,
                        method='Euler',
                        compile=True)

In [24]:
neurons_e.V = V_reset + np.random.uniform(1.1, 1.2, N_e) * (V_th-V_reset)
neurons_i.V = V_reset + np.random.uniform(1., 1.05, N_i) * (V_th-V_reset)